<a href="https://colab.research.google.com/github/nobuhikosekiya/elasticseach-query-apm-analyze-sample/blob/main/Elastic_Query_APM_analysis_manual_instrumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q elasticsearch elastic-apm
from pprint import pprint
import os, time
from getpass import getpass
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk
import elasticapm

# 接続先のElasitcsearchを設定

In [ ]:
ELASTIC_CLOUD_ID = getpass("Elastic deployment Cloud ID")
ELASTIC_API_KEY = getpass("Elastic deployment API Key")
ELASTIC_URL = getpass("Elastic deployment Elasticsearch URL. No need if Cloud ID is provided.")
ELASTIC_USER = getpass("Elastic user. No need if API key is provided.")
ELASTIC_PASSWORD = getpass("Elastic password. No need if API key is provided.")


if ELASTIC_CLOUD_ID != '' and ELASTIC_API_KEY != '':
  es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER != '' and ELASTIC_PASSWORD != '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
    request_timeout=300
  )
elif ELASTIC_CLOUD_ID != '' and ELASTIC_USER != '' and ELASTIC_PASSWORD != '':
  es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER == '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    request_timeout=300
  )
else:
  print("env needs to set either ELASTIC_CLOUD_ID or ELASTIC_URL")


pprint(es.info()) # should return cluster info

Elastic deployment Cloud ID··········
Elastic deployment API Key··········
Elastic deployment Elasticsearch URL. No need if Cloud ID is provided.··········
Elastic user. No need if API key is provided.··········
Elastic password. No need if API key is provided.··········
ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': '5e6289c08b9e480daea7eda7cf903523', 'cluster_uuid': 'TNamOtbIQFG3BotZjeVenw', 'version': {'number': '8.11.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '76013fa76dcbf144c886990c6290715f5dc2ae20', 'build_date': '2023-12-05T10:03:47.729926671Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


# Initialize Python APM

In [ ]:
ELASTIC_APM_URL = getpass("Elastic deployment APM URL.")
ELASITC_APM_SECRET= getpass("Elastic deployment APM secret token.")

client = elasticapm.Client(service_name="Elastic_Query_APM_analysis_manual-instrumentation", server_url=ELASTIC_APM_URL, secret_token=ELASITC_APM_SECRET, transaction_sample_rate=1.0)
# elasticapm.instrument() ... Do not enable if you do not want to auto instrument Elasticsearch queries

Elastic deployment APM URL.··········
Elastic deployment APM secret token.··········


Stack (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, 

# テストしよう

In [ ]:
INDEX_NAME="Elastic_Query_APM_analysis_manual-instrumentation".lower()
INDEX_NAME

'elastic_query_apm_analysis_manual-instrumentation'

## INDEXの作成

In [ ]:
if es.indices.exists(index=INDEX_NAME):
    # If it exists, delete the index
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' deleted successfully.")
else:
    print(f"Index '{INDEX_NAME}' does not exist.")

es.indices.create(
  index=INDEX_NAME,
  settings={
      "index": {
          "number_of_shards": 1,
          "number_of_replicas": 0
      }
  }
)

Index 'elastic_query_apm_analysis_manual-instrumentation' deleted successfully.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'elastic_query_apm_analysis_manual-instrumentation'})

## ドキュメントの作成

In [ ]:
docs_json=[
    {"question": "iphoneのインストール手順について教えて"},
    {"question": "I want to know how to setup an iphone"},
    {"question": "iosのセットアップ手順"},
    {"question": "iphoneどうやって使い始めればいいの？"},
    {"question": "androidのセットアップ方法などについて知りたい"},
    {"question": "iphoneかっこいいよね"},
    {"question": "iPhone 14が2月に発売されるらしい"}
]

# Delete previous documents
if es.indices.exists(index=INDEX_NAME):
    es.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})

index_docs = []
for doc_json in docs_json:
    index_docs.append({
        "_index": INDEX_NAME,
        "_source": doc_json,
    })

helpers.bulk(es, index_docs, refresh=True)

(7, [])

## サーチ - 1

In [ ]:
i=0
j=0
k=0

In [ ]:
client.begin_transaction(transaction_type="app")
elasticapm.set_user_context(username="nobuhiko")

search_key_word="iPhone"
query={"match": {"question": search_key_word}}
query_user_input=search_key_word

# set labels to the current transaction
elasticapm.label(query=query, search_key_word=search_key_word, execution_count=j)
response = es.search(index=INDEX_NAME, query=query)
print(response.body)
hit_count=response['hits']['total']['value']
# set labels to the current transaction
elasticapm.label(hits=hit_count)

client.end_transaction(name="search-manual-instrument1", result="success")

i=i+1

## サーチ - 2

In [ ]:
client.begin_transaction(transaction_type="app")
elasticapm.set_user_context(username="nobuhiko")

search_key_word="Windows"
query={"match": {"question": search_key_word}}
query_user_input=search_key_word

# set labels to the current transaction
elasticapm.label(query=query, search_key_word=search_key_word, execution_count=j)
response = es.search(index=INDEX_NAME, query=query)
print(response.body)
hit_count=response['hits']['total']['value']
# set labels to the current transaction
elasticapm.label(hits=hit_count)

client.end_transaction(name="search-manual-instrument2", result="success")

j=j+1

## サーチ - 3 無効なクエリー

In [ ]:
client.begin_transaction(transaction_type="app")
elasticapm.set_user_context(username="nobuhiko")

try:
  search_key_word="iPhone"
  query={"fuzzzzzzzzy": {"question": search_key_word}}
  query_user_input=search_key_word

  # set labels to the current transaction
  elasticapm.label(query=query, search_key_word=search_key_word, execution_count=j)
  response = es.search(index=INDEX_NAME, query=query)
  print(response.body)
  hit_count=response['hits']['total']['value']
  # set labels to the current transaction
  elasticapm.label(hits=hit_count)
except:
  elasticapm.set_transaction_outcome("failure")
  client.capture_exception()
  raise
finally:
  client.end_transaction(name="search-manual-instrument3", result="failure")
  k=k+1

BadRequestError: ignored

## クリックカウントのトラック

In [ ]:
client.begin_transaction(transaction_type="app")
elasticapm.set_user_context(username="nobuhiko")

elasticapm.label(clicked_transaction_name="search-manual-instrument2", clicked_doc_id="uySqYYwBoRDJc2lL7RCw", clicked_search_key_word="iPhone")

client.end_transaction(name="click_count", result="success")